In [2]:
import requests
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import locale
locale.setlocale(locale.LC_ALL, 'ID')
import re
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import html
import json

## GET INDEKS LINK

In [300]:
def getIndeksLink(links, page, date=datetime.strftime(datetime.today(), '%Y-%m-%d')):
    """
    Untuk mengambil seluruh url 
    link pada indeks category tertentu
    date format : YYYY/mm/dd
    """
    print("page ", page)
    url = "https://tirto.id/indeks/"+str(page)+"?date="+date
    print(url)
    
    # Make the request and create the response object: response
    response = requests.get(url)
    # Extract HTML texts contained in Response object: html
    html = response.text
    # Create a BeautifulSoup object from the HTML: soup
    soup = BeautifulSoup(html, "html5lib")
    
    main_el = soup.find('div', attrs={'id':'__nuxt'})
    script = main_el.findNextSiblings()
    tes = script[0].text.replace('window.__NUXT__=', '')[:-1]
    json_mobil = json.loads(tes)
    
    link_articles = json_mobil['data'][0]['listarticle']
    if link_articles:
        for art in link_articles:
            link = ['https://tirto.id'+art['articleUrl'], '']
            links.append(link)

    el_page = soup.find('ul', class_="custom-pagination p-0 text-center mb-5 col-10 offset-1")
    if el_page:
        max_page = int(el_page.findAll('a')[-2].text)
            
        if page < max_page:
            links = getIndeksLink(links, page+1, date)
        
    return links

## GET DETAIL BERITA

In [310]:
def getDetailBerita(links):
    all_articles = []
    for link in links:
        articles = {}
        #link
        url = link[0]
        response = requests.get(url)
        html = response.text
        # Create a BeautifulSoup object from the HTML: soup
        soup = BeautifulSoup(html, "html5lib")

        #extract subcategory from breadcrumb
        bc = soup.find('ol', class_="breadcrumbs")
        if not bc:
            continue
        cat = bc.findAll('a')[-1].text
        #sub = bc.findAll('a')[-1].text
            
        #category
        articles['category'] = cat
        articles['subcategory'] = ""
        
        #article_url
        articles['url'] = url        

        #article
        intro = soup.find('article', class_="col-12 content-detail-holder my-4").findAll('div', class_="content-text-editor")[0]
        article = soup.find('article', class_="col-12 content-detail-holder my-4").findAll('div', class_="content-text-editor")[1]
        author = soup.find('article', class_="col-12 content-detail-holder my-4").findAll('div', class_="content-text-editor")[2]
                
        #extract date
        author_pubdate = soup.find('div', class_="col-md-6").text.replace('Oleh: ','')
        author_pubdate = author_pubdate.split(' - ')
        pubdate = author_pubdate[1]
        pubdate = pubdate.strip(' ')
        articles['pubdate'] = datetime.strftime(datetime.strptime(pubdate, "%d %B %Y"), "%Y-%m-%d %H:%M:%S") 

        articles['id'] = int(datetime.strptime(pubdate, "%d %B %Y").timestamp()) + len(url)
        #extract author
        credit = soup.find('div', class_="credit").findAll('span', class_="reporter-grup")
        reporter_sumber = credit[0].text.replace('Reporter: ','')
        author = credit[1].text.replace('Penulis: ','')
        editor = credit[2].text.replace('Editor: ','')
        articles['author'] = author
        
        #extract title
        title = soup.find('h1', class_="news-detail-title text-center animated zoomInUp my-3").text
        if ("foto" in title.lower()) or  "video" in title.lower(): 
            continue
        articles['title'] = title   

        #source
        articles['source'] = 'tirto.id'

        #extract comments count
        articles['comments'] = 0

        #extract tags
        tags = soup.find('meta', attrs={"name":"keywords"})['content']
        articles['tags'] = tags

        #extract images
        image = soup.findAll('meta', attrs={"name":"thumbnail"})[0]['content']
        articles['image'] = image
        
        #hapus link sisip
        for link in article.findAll('div'):
            link.decompose()
            
        #extract content
        detail = BeautifulSoup(article.decode_contents().replace('<br/>', ' '), "html5lib")
        content = re.sub(r'\n|\t|\b|\r','',detail.text)
            
        #articles['content'] = re.sub('google*','', content).strip(' ')
        articles['content'] = content
        #print('memasukkan berita id ', articles['id'])
        all_articles.append(articles)
    return all_articles

In [302]:
links = getIndeksLink([], 1, '2018-07-30')

page  1
https://tirto.id/indeks/1?date=2018-07-30
page  2
https://tirto.id/indeks/2?date=2018-07-30
page  3
https://tirto.id/indeks/3?date=2018-07-30
page  4
https://tirto.id/indeks/4?date=2018-07-30


In [303]:
links

[['https://tirto.id/pks-perjuangkan-salim-segaf\xa0sebagai-cawapres-prabowo-cQdM',
  ''],
 ['https://tirto.id/kemensos-klaim-sudah-tangani-krisis-pangan-di-maluku-tengah-cQdN',
  ''],
 ['https://tirto.id/sby-sebut-pertemuan-demokrat-dan-pks-adalah-nostalgia-cQdL',
  ''],
 ['https://tirto.id/garuda-akan-evaluasi-target-laba-2018-menyusul-kenaikan-bahan-bakar-cQdF',
  ''],
 ['https://tirto.id/hasil-kejuaraan-dunia-2018-hari-ini-4-wakil-indonesia-ke-babak-dua-cQdD',
  ''],
 ['https://tirto.id/kerugian-garuda-indonesia-semester-i-2018-menyusut-5855-persen-cQdC',
  ''],
 ['https://tirto.id/bpjs-kesehatan-pastikan-tiga-aturan-baru-tetap-berjalan-cQdB',
  ''],
 ['https://tirto.id/hasil-jadwal-dan-klasemen-icc-2018-hingga-30-juli-cQdy',
  ''],
 ['https://tirto.id/gempa-lombok-386-pendaki-gunung-rinjani-berhasil-dievakuasi-cQdz',
  ''],
 ['https://tirto.id/klasemen-terbaru-liga-1-hingga-30-juli-persib-kokoh-di-puncak-cQdx',
  ''],
 ['https://tirto.id/hasil-ps-tira-vs-persib-skor-akhir-2-3-bauma

In [311]:
details = getDetailBerita(links[:3])
details

[{'author': 'Lalu Rahadian',
  'category': 'Politik',
  'comments': 0,
  'content': 'tirto.id - Partai Keadilan Sejahtera (PKS) akan memperjuangkan hasil Ijtimak Ulama yang merekomendasikan dua nama calon wakil presiden untuk mendampingi Ketua Umum Gerindra Prabowo Subianto di pemilu 2019.  Sekretaris Jenderal PKS Mustafa Kamal berkata, rekomendasi Ijtimak Ulama adalah aspirasi yang memperkuat koalisi partainya dengan Gerindra selama ini. Pernyataan itu ia sampaikan karena salah satu nama yang direkomendasikan Ijtimak Ulama menjadi cawapres adalah Ketua Majelis Syuro PKS Salim Segaf Al-Jufri.  "Ya itu jelas [memperjuangkan rekomendasi Ijtimak Ulama]. Kami partai yang dilahirkan oleh para ulama [...] Terbentuknya PKS tidak pernah lepas dari para ulama dan dukungan umat," tutur Mustafa di kawasan Kuningan, Jakarta, Senin (30/7/2018) malam.  Nama lain yang direkomendasikan Ijtimak Ulama menjadi cawapres Prabowo adalah Abdul Somad Batubara. Akan tetapi, Abdul Somad sudah mengaku tak ingin 

In [312]:
df = pd.DataFrame(details)

In [313]:
df

,author,category,comments,content,id,image,pubdate,source,subcategory,tags,title,url
0,Lalu Rahadian,Politik,0,tirto.id - Partai Keadilan Sejahtera (PKS) aka...,1532883674,https://mmc.tirto.id/image/otf/600x315/2018/07...,2018-07-30 00:00:00,tirto.id,,"pilpres 2019, pilpres 2019, cawapres 2019, caw...",PKS Perjuangkan Salim Segaf Sebagai Cawapres P...,https://tirto.id/pks-perjuangkan-salim-segaf s...
1,Damianus Andreas,Sosial Budaya,0,tirto.id - Kementerian Sosial (Kemensos) mengk...,1532883681,https://mmc.tirto.id/image/otf/600x315/2018/01...,2018-07-30 00:00:00,tirto.id,,"krisis pangan, krisis pangan, kelaparan, angka...",Kemensos Klaim Sudah Tangani Krisis Pangan di ...,https://tirto.id/kemensos-klaim-sudah-tangani-...
2,Lalu Rahadian,Politik,0,tirto.id - Pertemuan pengurus DPP Partai Demok...,1532883675,https://mmc.tirto.id/image/otf/600x315/2018/02...,2018-07-30 00:00:00,tirto.id,,"pilpres 2019, pilpres 2019, susilo bambang yud...",SBY Sebut Pertemuan Demokrat dan PKS adalah No...,https://tirto.id/sby-sebut-pertemuan-demokrat-...
